# Explore h5ad files from TAHOE-100M using Scanpy
#### Author: Leonardo Agasso, University of Turin (email: leonardo.agasso@unito.it)
___________________________________________________________________________________________

Import libraries

In [1]:
import pandas as pandas
import numpy as np 
import matplotlib.pyplot as plt
import scanpy as sc
import os
import sys

Here you need to define the relative path to the file you want to open. In this case the file name indicates that the cells come from Plate 1, the cancer cell-line is "HEC-1-A" (human endometrial adenocarcinoma), only protein-coding genes were filtered and 15 random drugs are taken into account.

In [6]:
n_plate = 1
cell_line = 'HEC-1-A'
biotype = 'protein_coding'

h5ad_file_path = f'../../dataset/plate{str(n_plate)}_cell_line_{cell_line}.{biotype}.15_rnd_drugs.h5ad'

Scanpy has a module (*sc.read_h5ad*) that allows to read h5ad files and store them in a specific object called *AnnData* (adata for brevity). Annotation data is the standard format used by Scanpy to store single-cell data.

In [8]:
adata = sc.read_h5ad(h5ad_file_path)
adata.shape

(36320, 19915)

Our file is quite large (470 Mb) but the annotation data handle large matrices very efficiently. The *shape* attribute of the *adata* object shows the number of cells and genes in the dataset, in this case we have 36,320 cells (*observables* in scanpy) and 19,915 genes (*variables* in scanpy).

We can take a look at what these genes and cells are stored in the *adata* object. Originally the data from the Tahoe-100M project didn't have the biotype, I labeled the genes according to the latest release of [GENCODE](https://www.gencodegenes.org/human/) in order to filter only protein-coding genes. A gene is typically adressed by its Gene Name (that is standardized by the HUGO Gene Nomenclature Committee; examples are TSPAN6, HOXB3 etc.) and/or by its Ensembl ID (a univocal identifier like ENSG00000291316, ENSG00000291313 etc.).

In [9]:
adata.var

,biotype
gene_name,
TSPAN6,protein_coding
TNMD,protein_coding
DPM1,protein_coding
SCYL3,protein_coding
FGR,protein_coding
...,...
ENSG00000291313,protein_coding
ENSG00000291314,protein_coding
ENSG00000291315,protein_coding


Cells have much more information in this dataset. The authors of the original work provide a table to understand the meaning of each column.

In [10]:
adata.obs

,sample,gene_count,tscp_count,mread_count,drugname_drugconc,drug,cell_line,sublibrary,BARCODE,pcnt_mito,S_score,G2M_score,phase,pass_filter,cell_name,plate
BARCODE_SUB_LIB_ID,,,,,,,,,,,,,,,,
06_008_135-lib_841,smp_1500,1830,2929,3481,"[('Capivasertib', 0.05, 'uM')]",Capivasertib,CVCL_0293,lib_841,06_008_135,0.136224,-0.119048,-0.094322,G1,full,HEC-1-A,plate1
06_009_187-lib_841,smp_1500,995,1330,1573,"[('Capivasertib', 0.05, 'uM')]",Capivasertib,CVCL_0293,lib_841,06_009_187,0.083459,0.071429,-0.080586,S,full,HEC-1-A,plate1
06_015_001-lib_841,smp_1500,3320,6043,7075,"[('Capivasertib', 0.05, 'uM')]",Capivasertib,CVCL_0293,lib_841,06_015_001,0.088201,-0.061905,-0.136081,G1,full,HEC-1-A,plate1
06_031_041-lib_841,smp_1500,1637,2373,2805,"[('Capivasertib', 0.05, 'uM')]",Capivasertib,CVCL_0293,lib_841,06_031_041,0.093552,0.019048,0.068498,G2M,full,HEC-1-A,plate1
06_031_049-lib_841,smp_1500,679,852,1029,"[('Capivasertib', 0.05, 'uM')]",Capivasertib,CVCL_0293,lib_841,06_031_049,0.103286,0.023810,-0.009158,S,full,HEC-1-A,plate1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93_183_069-lib_912,smp_1587,2377,3829,4561,"[('Anastrozole', 0.05, 'uM')]",Anastrozole,CVCL_0293,lib_912,93_183_069,0.068686,0.251196,-0.075727,S,full,HEC-1-A,plate1
93_185_090-lib_912,smp_1587,846,1100,1277,"[('Anastrozole', 0.05, 'uM')]",Anastrozole,CVCL_0293,lib_912,93_185_090,0.087273,-0.005013,0.024291,G2M,full,HEC-1-A,plate1
93_187_075-lib_912,smp_1587,958,1308,1563,"[('Anastrozole', 0.05, 'uM')]",Anastrozole,CVCL_0293,lib_912,93_187_075,0.074924,-0.048075,-0.038001,G1,full,HEC-1-A,plate1


Here's the table formatted with consistent spacing, copied from the [official repository](https://github.com/ArcInstitute/arc-virtual-cell-atlas/blob/main/tahoe-100/README.md?plain=1) of the project.

| Column Name            | Description                                                            |
|------------------------|------------------------------------------------------------------------|
| **plate**              | Plate identifier                                                       |
| **BARCODE_SUB_LIB_ID** | Cell identifier                                                        |
| **sample**             | Unique treatment identifier, distinguishes replicated treatments       |
| **gene_count**         | Number of genes with at least one count                                |
| **tscp_count**         | Number of transcripts, aka UMI count                                   |
| **mread_count**        | Number of reads per cell                                               |
| **drugname_drugconc**  | Drug name, concentration, and concentration unit                       |
| **drug**               | Drug name, parsed out from the `drugname_drugconc` field               |
| **cell_line**          | Cell line Cellosaurus identifier                                       |
| **sublibrary**         | Sublibrary ID (related to library prep and sequencing)                 |
| **BARCODE**            | Barcode ID                                                             |
| **pcnt_mito**          | Percentage of mitochondrial reads                                      |
| **S_score**            | Inferred S phase score                                                 |
| **G2M_score**          | Inferred G2M score                                                     |
| **phase**              | Inferred cell cycle phase                                              |
| **pass_filter**        | "Full" filters are more stringent on `gene_count` and `tscp_count`     |
| **cell_name**          | Commonly-used cell name (related to the `cell_line` field)             |
